In [1]:
import urllib.request
import itertools
import pandas as pd
import numpy as np
import json
import os

# Obtendo informação do tempo de transporte

Busca de informação em API de roteamento disponíveis na internet. No caso usando a API https://api.tomtom.com/

In [2]:
KEY = '<YOUR KEY HERE>'
#Chave da API do TomTom

def _request_json(url):
    req = urllib.request.Request(url)
    with urllib.request.urlopen(req) as response:
       content = response.read()
    return json.loads(content)


def _get_time_by_lat_long(o_json, d_json):
    '''
    Get time of travel by car with two jsons as input.
    Ex of json:{'lat': -19.91818, 'lon': -43.93705}
    '''
    try:
        prefix = 'https://api.tomtom.com/routing/1/calculateRoute/'
        origem = str(o_json['lat']) + ',' + str(o_json['lon']) #'52.50931,13.42936'
        dest = str(d_json['lat']) + ',' + str(d_json['lon']) #'52.50274,13.43872'

        complement = '/json?avoid=unpavedRoads&key='
        url = prefix + origem + ':' + dest + complement + KEY
        answer = _request_json(url)
        return answer['routes'][0]['summary']['travelTimeInSeconds']
    except Exception as msg:
        print('ERROR: ', msg)
        print('ORIGEM: ', origem)
        print('DESTINO: ', dest)
        print('URL: ', url)

def _get_lat_long_by_city_name(city_name, country='Brazil'):
    prefix = 'https://api.tomtom.com/search/2/geocode/'
    local = city_name.replace(' ','%20').replace('ã','a').replace('í','i').replace('á','a').replace('ú','u') #'Belo%20Horizonte'
    complement = '.json?key='

    url = prefix + local + complement + KEY
    answer = _request_json(url)
    #return answer['results'][0]['position']
    return next(item['position'] for item in answer['results'] if item['address']['country'] == country)

def get_time_by_origen_dest_names(o, d):
    pos_o = _get_lat_long_by_city_name(o)
    pos_d = _get_lat_long_by_city_name(d)
    return _get_time_by_lat_long(pos_o, pos_d)

In [3]:
#testing
print(get_time_by_origen_dest_names('Belo%20Horizonte', 'Curitiba'))

46533


## Construindo matrix com todas as capitais

In [14]:
name_dist_pickle = 'dist_capitais.pickle'

list_capitais = ['Sao Paulo','Manaus','Rio Branco','Campo Grande','Macapa','Brasilia','Boa Vista',
                 'Cuiaba','Palmas','Porto Velho','Teresina','Rio de Janeiro','Belem','Goiania',
                 'Salvador','Florianopolis','Sao Luis','Maceio','Porto Alegre','Curitiba',
                 'Belo Horizonte','Fortaleza','Recife','Joao Pessoa','Aracaju','Natal',
                 'Vitoria']
#Se já serializsou uma vez não precisa calcular novamente
if not os.path.isfile(name_dist_pickle):
    pair_list = []
    for pair in itertools.combinations(list_capitais, 2):
        print(pair)
        pair_list.append(pair + (get_time_by_origen_dest_names(*pair),))
        
    df = pd.DataFrame(data=pair_list,columns=['Origem','Destino','Tempo(s)'])
    df['Tempo(s)'] = df['Tempo(s)'].astype('double')
    df.to_pickle(name_dist_pickle)
else:
    df = pd.read_pickle(name_dist_pickle)

In [15]:
df.loc[df['Tempo(s)'].isnull()]
#Aparentemente a API não conseguiu calcular algumas rotas, vamos seguir como uma restrição de viabilidade

,Origem,Destino,Tempo(s)
51,Rio Branco,Campo Grande,NaN
53,Rio Branco,Brasilia,NaN
56,Rio Branco,Palmas,NaN
57,Rio Branco,Sao Paulo,NaN
59,Rio Branco,Rio de Janeiro,NaN
61,Rio Branco,Goiania,NaN
62,Rio Branco,Salvador,NaN
63,Rio Branco,Florianopolis,NaN
65,Rio Branco,Maceio,NaN
66,Rio Branco,Porto Alegre,NaN


In [16]:
df.fillna(value=df['Tempo(s)'].max()*10, inplace=True)
#rotas não encontradas terão um custo muito grande

# Modelo

In [17]:
import pyomo.environ as pyo

In [18]:
model = pyo.ConcreteModel()
#Indexes for the cities

model.U = list_capitais[1:]
n = len(list_capitais)

#Se viajou de a para b então x(a,b)=True
model.x = pyo.Var(list_capitais, list_capitais, within=pyo.Binary, initialize=0)

model.u = pyo.Var(list_capitais, within=pyo.NonNegativeIntegers,bounds=(0,n-1))


def dist_cost(i, j):
    '''
    May return dist(i,j) or dist(j,i) depending on the data matrix
    '''
    xij = df[df.Origem.isin([i]) & df.Destino.isin([j])]
    xji = df[df.Origem.isin([j]) & df.Destino.isin([i])]
    if xij.shape[0] > 0:
        return xij['Tempo(s)'].values[0]/3600 #em horas
    elif xji.shape[0] > 0:
        return xji['Tempo(s)'].values[0]/3600 #em horas
    else: #i==j
        return 0        
model.c = pyo.Param(list_capitais, list_capitais ,initialize=lambda model, i, j: dist_cost(i,j))

def destino_unico(model,M):
    return sum(model.x[i,M] for i in list_capitais if i!=M ) == 1
model.destino_unico = pyo.Constraint(list_capitais, rule=destino_unico)


def origem_unico(model,N):
    return sum(model.x[N,j] for j in list_capitais if j!=N) == 1
model.origem_unico = pyo.Constraint(list_capitais,rule=origem_unico)

def subtour(model,i,j):
    if i!=j: 
        return model.u[i] - model.u[j] + model.x[i,j] * n <= n-1
    else:
        return model.u[i] - model.u[i] == 0     
model.subtour = pyo.Constraint(model.U, list_capitais, rule=subtour)


def obj_func(model):
    return sum(model.x[i,j] * model.c[i,j] for i in list_capitais for j in list_capitais)

model.objective = pyo.Objective(rule=obj_func,sense=pyo.minimize)

In [37]:
solver = pyo.SolverFactory('gurobi')
solver.options['MIPGAP'] = 0.001
model.write('model.lp', io_options={'symbolic_solver_labels': True})

result = solver.solve(model, tee = True)

#Prints the results
print(result)

Using license file c:\gurobi\gurobi.lic
Set parameter ServerPassword
Set parameter TokenServer to value 52.184.165.91
Read LP format model from file C:\Users\RMANSUR\AppData\Local\Temp\tmplxyic8z3.pyomo.lp
Reading time = 0.03 seconds
x757: 757 rows, 730 columns, 3433 nonzeros
Changed value of parameter MIPGAP to 0.001
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 757 rows, 730 columns and 3433 nonzeros
Model fingerprint: 0x2bbcecce
Variable types: 1 continuous, 729 integer (702 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [2e+00, 8e+02]
  Bounds range     [1e+00, 3e+01]
  RHS range        [1e+00, 3e+01]
Presolve removed 27 rows and 2 columns
Presolve time: 0.02s
Presolved: 730 rows, 728 columns, 3406 nonzeros
Variable types: 0 continuous, 728 integer (702 binary)
Found heuristic solution: 

In [38]:
l = list(model.x.keys())
for i in l:
    if model.x[i]() != 0:
        print(i,'--', model.x[i]())

('Sao Paulo', 'Rio de Janeiro') -- 1.0
('Manaus', 'Porto Velho') -- 1.0
('Rio Branco', 'Cuiaba') -- 1.0
('Campo Grande', 'Porto Alegre') -- 1.0
('Macapa', 'Boa Vista') -- 1.0
('Brasilia', 'Goiania') -- 1.0
('Boa Vista', 'Manaus') -- 1.0
('Cuiaba', 'Campo Grande') -- 1.0
('Palmas', 'Salvador') -- 1.0
('Porto Velho', 'Rio Branco') -- 1.0
('Teresina', 'Belem') -- 1.0
('Rio de Janeiro', 'Vitoria') -- 1.0
('Belem', 'Macapa') -- 1.0
('Goiania', 'Palmas') -- 1.0
('Salvador', 'Aracaju') -- 1.0
('Florianopolis', 'Curitiba') -- 1.0
('Sao Luis', 'Teresina') -- 1.0
('Maceio', 'Recife') -- 1.0
('Porto Alegre', 'Florianopolis') -- 1.0
('Curitiba', 'Sao Paulo') -- 1.0
('Belo Horizonte', 'Brasilia') -- 1.0
('Fortaleza', 'Sao Luis') -- 1.0
('Recife', 'Joao Pessoa') -- 1.0
('Joao Pessoa', 'Natal') -- 1.0
('Aracaju', 'Maceio') -- 1.0
('Natal', 'Fortaleza') -- 1.0
('Vitoria', 'Belo Horizonte') -- 1.0


In [39]:
ordem = []
ordem.append(('Rio de Janeiro',0)) #primeiro destino

l = list(model.u.keys())
for i in l:
    if model.u[i]() != 0:
        ordem.append((i,model.u[i]()))


In [40]:
pd.DataFrame(data=ordem,columns=['Capital','Ordem']).sort_values(by='Ordem')

,Capital,Ordem
0,Rio de Janeiro,0.0
26,Vitoria,1.0
20,Belo Horizonte,2.0
6,Brasilia,3.0
13,Goiania,4.0
9,Palmas,5.0
14,Salvador,6.0
24,Aracaju,7.0
17,Maceio,8.0
22,Recife,9.0
